In [1]:
# Используемые библиотеки
import yaml
import csv
from collections import defaultdict
import os

In [ ]:
# Считывание нахождения в директории
file_path_yaml = os.getenv("file_path_yaml")
file_path_csv = os.getenv("file_path_csv")

In [3]:
def load_yaml(file_path):
    '''Функция для чтения .yaml'''
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

In [4]:
def load_logs(file_path):
    '''Функция для чтения .csv'''
    logs = []
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            logs.append(row['node_id'])
    return logs

In [ ]:
def find_statistics_nodes(folders):
    '''Функция для поиска по узлу'''
    #Мноежство для хранения результатов
    statistics_nodes = set()
    #Функция для проверки текущей ссылки на link
    def traverse(node):
        if 'link' in node:
            return
        if node.get('name') == 'statistics':
            statistics_nodes.add(node)
            return
        #Рекурсивный обход дочерних узлов
        for key, child in node.get('values', {}).items():
            traverse(child)
    #Рекурсивный обход узлов
    for key, node in folders.items():
        traverse(node)
    return statistics_nodes

In [ ]:
def get_all_node_ids(node):
    '''Функция для получения id узла'''
    node_ids = set()
    #Функция для проверки отношения узла с link
    def collect_ids(node):
        if 'link' in node:
            return
        if 'values' in node:
            #Рекурсивный обход дочерних ключей
            for key, child in node['values'].items():
                node_ids.add(key)
                collect_ids(child)
    collect_ids(node)
    return node_ids

In [ ]:
def main(file_path_yaml, file_path_csv):
    '''Функция присвоения данных'''
    folders = load_yaml(file_path_yaml)
    logs = load_logs(file_path_csv)